In [104]:
import rdflib

Load prefixes

In [105]:
prefixes = """
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX mod: <https://w3id.org/mod#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX xml: <http://www.w3.org/XML/1998/namespace>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX qudt: <http://qudt.org/2.1/schema/qudt/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX vann: <http://purl.org/vocab/vann/>
PREFIX cdqos: <https://w3id.org/candil/qos#>
PREFIX dcterms: <http://purl.org/dc/terms/>
"""

Load ontology and sample data

In [106]:
g = rdflib.Graph()
g.parse("../ontology/ontology.ttl", format='turtle')
g.parse("../examples/rdf/chronos-tf-integration.ttl", format='turtle')
print(len(g))

243


## CQ1: What is the application server of the qos session?

In [107]:
q_res = g.query(prefixes + """
    SELECT *
    WHERE {
        ?session a cdqos:Session ;
            dcterms:identifier ?id ;
            cdqos:applicationServer ?appServer .
    }
""")

for row in q_res:
    print(f"Session: {row.id} App Server: {row.appServer}")

Session: ceb5e725-d8f6-4eb7-abba-a5ce2ce60ace App Server: https://6g-chronos.org/ApplicationServer/192.168.0.1/24


## CQ2: What is the IPv4 and scsAsId of the application server?

In [108]:
q_res = g.query(prefixes + """
    SELECT *
    WHERE {
        ?session a cdqos:Session ;
            cdqos:applicationServer ?appServer .
        ?appServer cdqos:ipAddress ?address ;
            cdqos:scsAsId ?scsAsId .
    }
""")

for row in q_res:
    print(f"Application Server: {row.appServer} IPv4 Address: {row.address} SCS/AS Id: {row.scsAsId}")

Application Server: https://6g-chronos.org/ApplicationServer/192.168.0.1/24 IPv4 Address: 192.168.0.1/24 SCS/AS Id: myNetapp


## CQ3: What is the duration of the qos session?

In [109]:
q_res = g.query(prefixes + """
    SELECT *
    WHERE {
        ?session a cdqos:Session ;
            dcterms:identifier ?id ;
            cdqos:duration ?duration .
        ?duration qudt:numericValue ?value ;
            qudt:unit ?unit .
    }
""")

for row in q_res:
    print(f"Session: {row.id} Duration time: {row.value} Unit: {row.unit}")

Session: ceb5e725-d8f6-4eb7-abba-a5ce2ce60ace Duration time: 100 Unit: http://qudt.org/vocab/unit/SEC


## CQ4: What is the user equipment of the qos session?

In [110]:
q_res = g.query(prefixes + """
    SELECT *
    WHERE {
        ?session a cdqos:Session ;
            dcterms:identifier ?id ;
            cdqos:userEquipment ?ue .
        ?ue cdqos:ipAddress ?address
    }
""")

for row in q_res:
    print(f"Session: {row.id} User Equipment: {row.address}")

Session: ceb5e725-d8f6-4eb7-abba-a5ce2ce60ace User Equipment: 10.0.0.1


## CQ5: Which qos profile was assigned to the qos session?

In [111]:
q_res = g.query(prefixes + """
    SELECT *
    WHERE {
        ?session a cdqos:Session ;
            dcterms:identifier ?id ;
            cdqos:assignedQoSProfile ?profile .
        ?profile dcterms:identifier ?profileId .
    }
""")

for row in q_res:
    print(f"Session: {row.id} QoS Profile: {row.profileId}")

Session: ceb5e725-d8f6-4eb7-abba-a5ce2ce60ace QoS Profile: QOS_S


## CQ6: Which network profile maps to the qos profile?

In [112]:
q_res = g.query(prefixes + """
    SELECT *
    WHERE {
        ?session a cdqos:Session ;
            dcterms:identifier ?id ;
            cdqos:assignedQoSProfile ?qosProfile .
        ?qosProfile cdqos:mapsToNetworkProfile ?networkProfile ;
            dcterms:identifier ?qosProfileId .
        ?networkProfile dcterms:identifier ?networkProfileId .
    }
""")

for row in q_res:
    print(f"Session: {row.id} QoS Profile: {row.qosProfileId} Network Profile: {row.networkProfileId}")

Session: ceb5e725-d8f6-4eb7-abba-a5ce2ce60ace QoS Profile: QOS_S Network Profile: 1


## CQ7: Which network profile is used by the network slice?

In [113]:
q_res = g.query(prefixes + """
    SELECT *
    WHERE {
        ?slice a cdqos:NetworkSlice ;
            dcterms:identifier ?id ;
            cdqos:usesNetworkProfile ?networkProfile .
        ?networkProfile dcterms:identifier ?networkProfileId .
    }
""")

for row in q_res:
    print(f"Network Slice: {row.id} Network Profile: {row.networkProfileId}")

Network Slice: slice_medium_performance_2 Network Profile: 1
Network Slice: slice_medium_performance_6 Network Profile: 1
Network Slice: slice_medium_performance_10 Network Profile: 1
Network Slice: slice_medium_performance_14 Network Profile: 1
Network Slice: slice_medium_performance_18 Network Profile: 1
Network Slice: slice_medium_performance_22 Network Profile: 1


## CQ8: Which 5QI are mapped to the network profile?

In [114]:
q_res = g.query(prefixes + """
    SELECT *
    WHERE {

        ?networkProfile a cdqos:NetworkProfile ;
            dcterms:identifier ?networkProfileId .
        ?networkSlice a cdqos:NetworkSlice ;
            cdqos:usesNetworkProfile ?networkProfile ;
            cdqos:5qi ?5qi .
        ?5qi dcterms:identifier ?qId .
    }
""")

for row in q_res:
    print(f"5QI: {row.qId} Network Profile: {row.networkProfileId}")

5QI: 2 Network Profile: 1
5QI: 27 Network Profile: 1
5QI: 1 Network Profile: 1
5QI: 8 Network Profile: 1
5QI: 1 Network Profile: 1
5QI: 8 Network Profile: 1


## CQ9: What is the MBR downlink of a network slice?

In [115]:
q_res = g.query(prefixes + """
    SELECT *
    WHERE {
        ?networkSlice a cdqos:NetworkSlice ;
            dcterms:identifier ?id ;
            cdqos:mbrDownlink ?mbrDownlink .
        ?mbrDownlink qudt:numericValue ?value ;
            qudt:unit ?unit .
    }
""")

for row in q_res:
    print(f"Network Slice: {row.id} MBR Downlink: {row.value} Unit: {row.unit}")

Network Slice: slice_medium_performance_2 MBR Downlink: 55 Unit: http://qudt.org/vocab/unit/MegaBIT-PER-SEC
Network Slice: slice_medium_performance_6 MBR Downlink: 55 Unit: http://qudt.org/vocab/unit/MegaBIT-PER-SEC
Network Slice: slice_medium_performance_10 MBR Downlink: 55 Unit: http://qudt.org/vocab/unit/MegaBIT-PER-SEC
Network Slice: slice_medium_performance_14 MBR Downlink: 55 Unit: http://qudt.org/vocab/unit/MegaBIT-PER-SEC
Network Slice: slice_medium_performance_18 MBR Downlink: 55 Unit: http://qudt.org/vocab/unit/MegaBIT-PER-SEC
Network Slice: slice_medium_performance_22 MBR Downlink: 55 Unit: http://qudt.org/vocab/unit/MegaBIT-PER-SEC


## CQ10: What is the MBR uplink of a network slice?

In [116]:
q_res = g.query(prefixes + """
    SELECT *
    WHERE {
        ?networkSlice a cdqos:NetworkSlice ;
            dcterms:identifier ?id ;
            cdqos:mbrUplink ?mbrUplink .
        ?mbrUplink qudt:numericValue ?value ;
            qudt:unit ?unit .
    }
""")

for row in q_res:
    print(f"Network Slice: {row.id} MBR Uplink: {row.value} Unit: {row.unit}")

Network Slice: slice_medium_performance_2 MBR Uplink: 22 Unit: http://qudt.org/vocab/unit/MegaBIT-PER-SEC
Network Slice: slice_medium_performance_6 MBR Uplink: 25 Unit: http://qudt.org/vocab/unit/MegaBIT-PER-SEC
Network Slice: slice_medium_performance_10 MBR Uplink: 25 Unit: http://qudt.org/vocab/unit/MegaBIT-PER-SEC
Network Slice: slice_medium_performance_14 MBR Uplink: 25 Unit: http://qudt.org/vocab/unit/MegaBIT-PER-SEC
Network Slice: slice_medium_performance_18 MBR Uplink: 25 Unit: http://qudt.org/vocab/unit/MegaBIT-PER-SEC
Network Slice: slice_medium_performance_22 MBR Uplink: 25 Unit: http://qudt.org/vocab/unit/MegaBIT-PER-SEC
